## Dataset loader and preparation

In [3]:
import sys
sys.path.insert(0, "../")
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import torch
 
data = pd.read_csv(r"./sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/time_series/dataAnalysis/data/Filter.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.data['Label'].mask(control_filter, "Control", inplace=True)
/home/dwalke/git/time_series/dataAnalysis/data/Filter.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [4]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
max_Id = data["Id"].unique().max()
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)
data = pd.concat((data, gw_data))
data = data.sort_values(["Id", "Time"])
data = data.reset_index(drop=True)
popped_index = data.pop("index")

In [5]:
from dataAnalysis.Constants import SEX_CATEGORY_COLUMN_NAME, SEX_COLUMN_NAME, FEATURES, LABEL_COLUMN_NAME

data[SEX_CATEGORY_COLUMN_NAME] = data.loc[:, SEX_COLUMN_NAME] =="W"

data[SEX_CATEGORY_COLUMN_NAME] = data[SEX_CATEGORY_COLUMN_NAME].astype("int8")
data["Label"] = data["Label"] == "Sepsis"
data["Label"] = data["Label"].astype("int8")

## Normalizer

In [6]:
data.loc[data["Set"] == "Training", FEATURES].values

array([[ 79.  ,   0.  ,   8.1 , ...,   4.36,  86.  , 167.  ],
       [ 35.  ,   0.  ,  10.6 , ...,   6.02,  79.9 , 199.  ],
       [ 47.  ,   1.  ,   8.7 , ...,   4.37,  89.9 , 298.  ],
       ...,
       [ 32.  ,   1.  ,   7.2 , ...,   3.87,  87.9 , 221.  ],
       [ 47.  ,   0.  ,   8.3 , ...,   4.08,  91.9 , 148.  ],
       [ 47.  ,   0.  ,   8.9 , ...,   4.39,  92.3 , 150.  ]])

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(data.loc[data["Set"] == "Training", FEATURES].values)

StandardScaler()

## Padding to max length

In [8]:
id_grouped_data = data.groupby("Id")

In [9]:
from tqdm.notebook import tqdm

max_len = 0
for id, data_group in tqdm(id_grouped_data):
    max_len = max(max_len, data_group.shape[0])

  0%|          | 0/866517 [00:00<?, ?it/s]

In [10]:
unique_train_ids = data.loc[data["Set"] == "Training", "Id"].unique()
max_train_idx = int(len(unique_train_ids)*.8)
val_ids = unique_train_ids[max_train_idx:]

In [11]:
import numpy as np

pad_value = 0
TRAIN = "train"
GW = "gw_test"
TEST = "test"
VAL = "val"

features = dict({})
features[TRAIN] = []
features[VAL] = []
features[TEST] = []
features[GW] = []

labels = dict({})
labels[TRAIN] = []
labels[VAL] = []
labels[TEST] = []
labels[GW] = []

for id, data_group in tqdm(id_grouped_data):
    assert data_group["Set"].unique().shape[0] == 1
    assert data_group["Center"].unique().shape[0] == 1

    features_scaled = scaler.transform(data_group[FEATURES].values)

    padded_features = np.pad(features_scaled, ((0, max_len - data_group.shape[0]), (0,0)), mode='constant', constant_values=pad_value)
    padded_labels = np.pad(data_group[LABEL_COLUMN_NAME].values, ((0, max_len - data_group.shape[0])), mode='constant', constant_values=pad_value)
    
    first_el = data_group.iloc[0, :]
    if first_el["Set"] == "Training":
        if first_el["Id"] in val_ids:
            features[VAL].append(padded_features)
            labels[VAL].append(padded_labels)
            continue
        if first_el["Id"] not in val_ids:
            features[TRAIN].append(padded_features)
            labels[TRAIN].append(padded_labels)
            continue
    if first_el["Set"] == "Validation":
        if first_el["Center"] == "Greifswald":
            features[GW].append(padded_features)
            labels[GW].append(padded_labels)
            continue
        if first_el["Center"] == "Leipzig":
            features[TEST].append(padded_features)
            labels[TEST].append(padded_labels)
            continue

  0%|          | 0/866517 [00:00<?, ?it/s]

## Seed and hyperparams

In [12]:
torch.manual_seed(42)
n_embd = len(FEATURES)
head_size = 16
dropout = 0
out_dim = 1 #binary 
device = torch.device("cuda:0") #torch.device("cuda:2") #torch.device("cuda:2")
WEIGHT = 664
lr = 1e-2
wd = 0
n_blocks = 2
n_heads = 4

## Batching 

In [13]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

class Data(Dataset):
    # Constructor
    def __init__(self,X, y):
        self.x = X
        self.y = y
        self.len = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.len

In [14]:
batch_loader = dict({})

In [15]:
def add_loader(set_name):
    global bitch_loader, sets, labels
    dataset = Data(torch.from_numpy(np.array(features[set_name])).type(torch.float).to(device), torch.from_numpy(np.array(labels[set_name])).type(torch.float).to(device))
    loader = DataLoader(dataset=dataset, batch_size=500) #max is 100_000
    batch_loader[set_name] = loader

In [16]:
add_loader(TRAIN)
add_loader(VAL)
add_loader(GW)
add_loader(TEST)

## Model

In [17]:
from torch import nn
from torch.nn import functional as F

torch.manual_seed(42)
class ConvModel(nn.Module):

    def __init__(self, input_dim, fiter_size, fiter_size_2,fiter_size_3):
        super(ConvModel, self).__init__()

        self.pos_embedding_table = nn.Embedding(max_len, input_dim)
        
        self.conv = nn.Conv1d(input_dim, 1, fiter_size)
        self.conv2 = nn.Conv1d(input_dim, 1, fiter_size_2)
        self.conv3 = nn.Conv1d(input_dim, 1, fiter_size_3)
        self.lin_heads = nn.Linear(3, 1)

    def forward(self, x, targets = None):
        B, T, C = x.shape
        pad_mask = (x != pad_value).type(torch.float)
        ignore_mask = torch.bmm(pad_mask, pad_mask.transpose(-2,-1))
        ignore_mask = ignore_mask == 0
        
        # pos_emb = self.pos_embedding_table(torch.arange(T, device = device)) ##destroys zero pad
        # x = x + pos_emb
        origin_x = torch.clone(x)
        
        conv_x = torch.nn.functional.pad(x, (0,0,0,self.conv.kernel_size[0]-1,0,0), mode='constant', value=pad_value)
        conv2_x = torch.nn.functional.pad(x, (0,0,0,self.conv2.kernel_size[0]-1,0,0), mode='constant', value=pad_value)
        conv3_x = torch.nn.functional.pad(x, (0,0,0,self.conv3.kernel_size[0]-1,0,0), mode='constant', value=pad_value)
        
        conv_x = conv_x.permute(0, 2, 1)
        conv2_x = conv2_x.permute(0, 2, 1)
        conv3_x = conv3_x.permute(0, 2, 1)
        
        logits_conv1 = self.conv(conv_x)
        logits_conv2 = self.conv2(conv2_x)
        logits_conv3 = self.conv3(conv3_x)

        logits_conv1 = torch.nn.functional.elu(logits_conv1)
        logits_conv2 = torch.nn.functional.elu(logits_conv2)
        logits_conv3 = torch.nn.functional.elu(logits_conv3)
        
        conv_heads = torch.cat((logits_conv1, logits_conv2,logits_conv3), dim = -2)
        logits = self.lin_heads(conv_heads.permute(0,2,1))
                
        logits_mask = torch.logical_not(torch.all(ignore_mask, dim =-1))
        return logits.squeeze(-1), logits_mask

In [26]:
from torch import nn
from torch.nn import functional as F

torch.manual_seed(42)
class ConvModel(nn.Module):

    def __init__(self, input_dim, filters):
        super(ConvModel, self).__init__()

        self.pos_embedding_table = nn.Embedding(max_len, input_dim)

        self.convs = nn.ModuleList()
        self.filters = filters
        for filter in filters:
            self.convs.append(nn.Conv1d(input_dim, 1, filter).to(device))
        self.lin_heads =nn.Linear(len(filters), 1)

    def forward(self, x, targets = None):
        B, T, C = x.shape
        pad_mask = (x != pad_value).type(torch.float)
        ignore_mask = torch.bmm(pad_mask, pad_mask.transpose(-2,-1))
        ignore_mask = ignore_mask == 0
        
        # pos_emb = self.pos_embedding_table(torch.arange(T, device = device)) ##destroys zero pad
        # x = x + pos_emb
        origin_x = torch.clone(x)

        conv_logits = []
        for i, filter in enumerate(self.filters):
            conv_x = torch.nn.functional.pad(x, (0,0,0,self.convs[i].kernel_size[0]-1,0,0), mode='constant', value=pad_value)
            
            conv_x = conv_x.permute(0, 2, 1)

            conv = self.convs[i]
            logits_conv1 = conv(conv_x)
    
            logits_conv1 = torch.nn.functional.elu(logits_conv1)
            conv_logits.append(logits_conv1)

        conv_heads = torch.cat(conv_logits, dim = -2)
        logits = self.lin_heads(conv_heads.permute(0,2,1))
                
        logits_mask = torch.logical_not(torch.all(ignore_mask, dim =-1))
        return logits.squeeze(-1), logits_mask

In [42]:
from torch import nn
from torch.nn import functional as F

torch.manual_seed(42)
class ConvModel(nn.Module):

    def __init__(self, input_dim, filter_size):
        super(ConvModel, self).__init__()

        self.pos_embedding_table = nn.Embedding(max_len, input_dim)

        self.conv = nn.Conv1d(input_dim, 5, filter_size).to(device)
        self.lin_heads =nn.Linear(5, 1)

    def forward(self, x, targets = None):
        B, T, C = x.shape
        pad_mask = (x != pad_value).type(torch.float)
        ignore_mask = torch.bmm(pad_mask, pad_mask.transpose(-2,-1))
        ignore_mask = ignore_mask == 0
        
        # pos_emb = self.pos_embedding_table(torch.arange(T, device = device)) ##destroys zero pad
        # x = x + pos_emb
        origin_x = torch.clone(x)

        conv_logits = []
        conv_x = torch.nn.functional.pad(x, (0,0,0,self.conv.kernel_size[0]-1,0,0), mode='constant', value=pad_value)
        
        conv_x = conv_x.permute(0, 2, 1)

        logits_conv1 = self.conv(conv_x)

        logits_conv1 = torch.nn.functional.elu(logits_conv1)

        logits = self.lin_heads(logits_conv1.permute(0,2,1))
                
        logits_mask = torch.logical_not(torch.all(ignore_mask, dim =-1))
        return logits.squeeze(-1), logits_mask

In [43]:
input_dim = len(FEATURES)

In [24]:
model = ConvModel(input_dim, 5).to(device)
evaluate_loss_sets(model)

/home/dwalke/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Loss of train: 0.00299
Loss of val: 0.00313
Loss of test: 0.00281
Loss of gw_test: 0.00239


{'train': 0.002989539451850587,
 'val': 0.0031329219213356726,
 'test': 0.002810940051504926,
 'gw_test': 0.0023891535918084965}

## Evaluate

In [44]:
def evaluate_loss(model, set_name):
    with torch.inference_mode():
        model.eval()
        acc_loss = 0
        batch_size = 0

        for i, (x,y) in enumerate(batch_loader[set_name]):
            B,_,_ = x.shape
            logits, logits_mask = model(x)
        
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits[logits_mask].squeeze(-1), y[logits_mask], pos_weight=torch.tensor(WEIGHT))
            acc_loss += loss.item()
            batch_size += B
    return acc_loss / batch_size        

In [45]:
def evaluate_loss_sets(model):
    losses = dict({})
    for set_name in features.keys():
        # if set_name != VAL:
        #     continue
        loss = evaluate_loss(model, set_name)
        losses[set_name] = loss
        print(f"Loss of {set_name}: {loss:.5f}") 
    return losses
        

In [46]:
from sklearn.metrics import roc_auc_score, accuracy_score

def evaluate_auroc(model, set_name):
    with torch.inference_mode():
        model.eval()
        
        logits_list = []
        label_list = []

        batch_size = 0
        for i, (x,y) in enumerate(batch_loader[set_name]):
            B,_,_ = x.shape
            logits, logits_mask = model(x)

            logits_list.extend(logits[logits_mask].squeeze(-1).tolist())
            label_list.extend(y[logits_mask].squeeze(-1).tolist())
            batch_size += B
    auroc = roc_auc_score(np.array(label_list), torch.sigmoid(torch.tensor(logits_list)).numpy())
    return auroc        

In [47]:
def evaluate_auroc_sets(model):
    for set_name in features.keys():
        if set_name == TRAIN:
            continue
        auroc = evaluate_auroc(model, set_name)
        print(f"AUROC of {set_name}: {auroc:.5f}")

## Train

In [48]:
torch.manual_seed(42)
input_dim, n_embd, n_heads, dropout, n_blocks = len(FEATURES), len(FEATURES), 7, 0.0, 1
model = ConvModel(input_dim, 5).to(device)
optim= torch.optim.Adam(model.parameters(), lr = 1e-2, weight_decay=wd)

In [49]:
torch.manual_seed(42)

evaluate_loss_sets(model)
last_val_loss = None
for epoch in range(65):#5
    for i, (x,y) in tqdm(enumerate(batch_loader[TRAIN])):
        model.train()
        optim.zero_grad()
        
        logits, logits_mask = model(x)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logits[logits_mask].squeeze(-1), y[logits_mask], pos_weight=torch.tensor(WEIGHT))
            
        loss.backward()
        optim.step()
    losses = evaluate_loss_sets(model)
    if last_val_loss and last_val_loss <= losses[VAL]:
        print(epoch)
        break
    last_val_loss = losses[VAL]

Loss of train: 0.00279
Loss of val: 0.00288
Loss of test: 0.00269
Loss of gw_test: 0.00250


0it [00:00, ?it/s]

Loss of train: 0.00127
Loss of val: 0.00136
Loss of test: 0.00124
Loss of gw_test: 0.00105


0it [00:00, ?it/s]

Loss of train: 0.00121
Loss of val: 0.00131
Loss of test: 0.00120
Loss of gw_test: 0.00099


...

Loss of train: 0.00109
Loss of val: 0.00122
Loss of test: 0.00112
Loss of gw_test: 0.00096


0it [00:00, ?it/s]

Loss of train: 0.00109
Loss of val: 0.00122
Loss of test: 0.00112
Loss of gw_test: 0.00096


0it [00:00, ?it/s]

Loss of train: 0.00109
Loss of val: 0.00122
Loss of test: 0.00112
Loss of gw_test: 0.00096
27


In [50]:
torch.manual_seed(42)
evaluate_auroc_sets(model)

AUROC of val: 0.95227
AUROC of test: 0.95103
AUROC of gw_test: 0.95006


In [395]:
torch.manual_seed(42)
evaluate_auroc_sets(model)#0876,88187,88417,0.88449,0.88569,,0.88623,,0.88692

AUROC of val: 0.95414
AUROC of test: 0.95012
AUROC of gw_test: 0.95118


In [ ]:
from sklearn.metrics import confusion_matrix

def evaluate_confusion(model, set_name):
    with torch.inference_mode():
        model.eval()
        
        logits_list = []
        label_list = []

        batch_size = 0
        for i, (x,y) in enumerate(batch_loader[set_name]):
            B,_,_ = x.shape
            logits, logits_mask = model(x)

            logits_list.extend(logits[logits_mask].squeeze(-1).tolist())
            label_list.extend(y[logits_mask].squeeze(-1).tolist())
            batch_size += B
    conf = confusion_matrix(np.array(label_list), torch.round(torch.sigmoid(torch.tensor(logits_list))).numpy())
    return conf

In [ ]:
evaluate_confusion(model, TEST)

In [32]:
from sklearn.model_selection import ParameterGrid
import copy

space = {
    'f1': [i for i in range(2,6)],
    'f2': [i for i in range(2,6)],
    'f3': [i for i in range(2,6)],
    'f4': [i for i in range(2,6)],
    'f5': [i for i in range(2,6)],
}
param_grid = ParameterGrid(space)

best_hyper_params = {'f1': 3, 'f2': 4, 'f3': 5, 'f4': 3, 'f5': 5}
best_val_loss = 0.0011606613010549842

processed_params = []
processed_losses = []

save_progress_iter = 2

patience = 2
for i, params in tqdm(enumerate(param_grid.__iter__())):
    if i <= 913:
        continue    
    
    torch.manual_seed(42)
    model = ConvModel(input_dim, [params[f"f{i}"] for i in range(1,6)]).to(device)
    optim= torch.optim.Adam(model.parameters(), lr = 1e-2)
    models = []

    last_val_loss = None
    val_losses = []
    never_breaked = True
    for epoch in range(100):
        for i, (x,y) in tqdm(enumerate(batch_loader[TRAIN])):
            model.train()
            optim.zero_grad()
            
            logits, logits_mask = model(x)
            loss = torch.nn.functional.binary_cross_entropy_with_logits(logits[logits_mask].squeeze(-1), y[logits_mask], pos_weight=torch.tensor(WEIGHT))
                
            loss.backward()
            optim.step()
        losses = evaluate_loss_sets(model)
        val_losses.append(losses[VAL])
        models.append(copy.deepcopy(model))
        print(val_losses)
        if len(val_losses) >= patience and all(list(map(lambda l: losses[VAL] >= l, val_losses[-patience:]))):
            print(f"Break at {epoch}")
            val_losses = val_losses[:-patience]
            never_breaked = False
            model = models[-patience]
            break
    if never_breaked:
        print("Never breaked with params")
    processed_params.append(params)
    processed_losses.append(val_losses[-1])

    if i % save_progress_iter == 0:
        df = pd.DataFrame()
        df["params"] = processed_params
        df["loss"] = processed_losses
        df.to_csv("cnn_params.csv")
    
    # evaluate_auroc_sets(model)
    if val_losses[-1]<= best_val_loss:
        best_val_loss = min(val_losses[-1], best_val_loss)
        best_hyper_params = params
        print("Currently best params: ")
        print(best_hyper_params)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Loss of train: 0.00135
Loss of val: 0.00145
Loss of test: 0.00129
Loss of gw_test: 0.00112
[0.001454187366165176]


0it [00:00, ?it/s]

Loss of train: 0.00130
Loss of val: 0.00140
Loss of test: 0.00122
Loss of gw_test: 0.00108
[0.001454187366165176, 0.0013962066526764922]


...

Loss of train: 0.00111
Loss of val: 0.00123
Loss of test: 0.00112
Loss of gw_test: 0.00094
[0.0013575464129575476, 0.0013014083628326222, 0.0012908866416412406, 0.0012852397347222079, 0.0012743569613338254, 0.001263582200166614, 0.0012590461709277895, 0.0012571098960882387, 0.0012559615586091363, 0.0012547615549959638, 0.0012530984286088346, 0.001250627809564933, 0.001248123942002674, 0.0012454085569852768, 0.0012428875275730692, 0.0012409229216082311, 0.0012395407704477402, 0.001238733008043283, 0.0012381128284891252, 0.0012373687723897134, 0.0012364652354490554, 0.0012354489085647671, 0.0012344090305349534, 0.001233232830461976, 0.001231875389571999, 0.0012304688227190096, 0.0012288365611384802]


0it [00:00, ?it/s]

Loss of train: 0.00111
Loss of val: 0.00123
Loss of test: 0.00112
Loss of gw_test: 0.00095
[0.0013575464129575476, 0.0013014083628326222, 0.0012908866416412406, 0.0012852397347222079, 0.0012743569613338254, 0.001263582200166614, 0.0012590461709277895, 0.0012571098960882387, 0.0012559615586091363, 0.0012547615549959638, 0.0012530984286088346, 0.001250627809564933, 0.001248123942002674, 0.0012454085569852768, 0.0012428875275730692, 0.0012409229216082311, 0.0012395407704477402, 0.001238733008043283, 0.0012381128284891252, 0.0012373687723897134, 0.0012364652354490554, 0.0012354489085647671, 0.0012344090305349534, 0.001233232830461976, 0.001231875389571999, 0.0012304688227190096, 0.0012288365611384802, 0.001228546465279876]


0it [00:00, ?it/s]

Loss of train: 0.00111
Loss of val: 0.00123
Loss of test: 0.00112
Loss of gw_test: 0.00095
[0.0013575464129575476, 0.0013014083628326222, 0.0012908866416412406, 0.0012852397347222079, 0.0012743569613338254, 0.001263582200166614, 0.0012590461709277895, 0.0012571098960882387, 0.0012559615586091363, 0.0012547615549959638, 0.0012530984286088346, 0.001250627809564933, 0.001248123942002674, 0.0012454085569852768, 0.0012428875275730692, 0.0012409229216082311, 0.0012395407704477402, 0.001238733008043283, 0.0012381128284891252, 0.0012373687723897134, 0.0012364652354490554, 0.0012354489085647671, 0.0012344090305349534, 0.001233232830461976, 0.001231875389571999, 0.0012304688227190096, 0.0012288365611384802, 0.001228546465279876, 0.0012285957429054202]
Break at 28


In [ ]:
raise Exception

In [33]:
best_hyper_params

{'f1': 3, 'f2': 4, 'f3': 5, 'f4': 3, 'f5': 5}

In [34]:
best_val_loss

0.0011606613010549842